# Exercises for Chapter 4

### Imputing missing data

Load the [Pima diabetes dataset]('http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data') as a pandas dataframe. (Note that the data does not include a header row. You'll have to build that yourself based on the documentation.)

In [61]:
import pandas

names = ['num_times_pregnant', 'glucose_concentration',
          'blood_pressure', 'skin_fold_thickness', 'insulin',
          'bmi', 'diabetes_pedigree', 'age', 'target']

data_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/' +\
           'pima-indians-diabetes/pima-indians-diabetes.data'

df = pandas.read_csv(data_url, header=None, index_col=False, names=names)
print(df)

     num_times_pregnant  glucose_concentration  blood_pressure  \
0                     6                    148              72   
1                     1                     85              66   
2                     8                    183              64   
3                     1                     89              66   
4                     0                    137              40   
5                     5                    116              74   
6                     3                     78              50   
7                    10                    115               0   
8                     2                    197              70   
9                     8                    125              96   
10                    4                    110              92   
11                   10                    168              74   
12                   10                    139              80   
13                    1                    189              60   
14        

Check the dataframe to see which columns contain 0's. Based on the data type of each column, do these 0's all make sense? Which 0's are suspicious?

In [62]:
for name in names:
    print(name, ':', any(df.loc[:, name] == 0))

num_times_pregnant : True
glucose_concentration : True
blood_pressure : True
skin_fold_thickness : True
insulin : True
bmi : True
diabetes_pedigree : False
age : False
target : True


**Answer**: Columns 2-6 (glucose, blood pressure, skin fold thickness, insulin, and BMI) all contain zeros, but none of these measurements should ever be 0 in a human.

Assume that 0s indiciate missing values, and fix them in the dataset by eliminating samples with missing features. Then run a logistic regression, and measure the performance of the model.

In [63]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

for i in range(1,6):
    df.loc[df.loc[:, names[i]] == 0, names[i]] = np.nan

df_no_nan = df.dropna(axis=0, how='any')

X = df_no_nan.iloc[:, :8].values
y = df_no_nan.iloc[:, 8].values

def fit_and_score_rlr(X, y, normalize=True):
    
    if normalize:
        scaler = StandardScaler().fit(X)
        X_std = scaler.transform(X)
    else:
        X_std = X
        
    X_train, X_test, y_train, y_test = train_test_split(X_std, y,
                                                        test_size=0.33,
                                                        random_state=42)

    rlr = LogisticRegression(C=1)

    rlr.fit(X_train, y_train)
    return rlr.score(X_test, y_test)

fit_and_score_rlr(X, y)

0.7384615384615385

Next, replace missing features through mean imputation. Run  a regression and measure the performance of the model.

In [64]:
from sklearn.preprocessing import Imputer

imputer = Imputer(missing_values='NaN', strategy='mean', axis=1)
X = imputer.fit_transform(df.iloc[:, :8].values)

y = df.iloc[:, 8].values

fit_and_score_rlr(X, y)

0.76377952755905509

Comment on your results.

**Answer**: Interestingly, there's not a huge performance improvement between the two approaches! In my run, using mean imputation corresponded to about a 3 point increase in model performance. Some ideas for why this might be:

1. This is a small dataset to start out with, so removing ~half its samples doesn't change performance very much
2. There's not much information contained in the features with missing data
3. There are other effects underlying poor performance of the model (e.g. regularization parameters) that are having a greater impact

### Preprocessing categorical variables

Load the [TA evaluation dataset](https://archive.ics.uci.edu/ml/datasets/Teaching+Assistant+Evaluation). As before, the data and header are split into two files, so you'll have to combine them yourself.

In [65]:
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/tae/tae.data'

names = ['native_speaker', 'instructor', 'course', 'season', 'class_size', 'rating']

df = pandas.read_csv(data_url, header=None, index_col=False, names=names)
print(df)

     native_speaker  instructor  course  season  class_size  rating
0                 1          23       3       1          19       3
1                 2          15       3       1          17       3
2                 1          23       3       2          49       3
3                 1           5       2       2          33       3
4                 2           7      11       2          55       3
5                 2          23       3       1          20       3
6                 2           9       5       2          19       3
7                 2          10       3       2          27       3
8                 1          22       3       1          58       3
9                 2          15       3       1          20       3
10                2          10      22       2           9       3
11                2          13       1       2          30       3
12                2          18      21       2          29       3
13                2           6      17       2 

Which of the features are categorical? Are they ordinal, or nominal? Which features are numeric?

**Answer**: According to the documentation:

1. **Native speaker**: categorical (nominal)
2. **Instructor**: categorical (nominal)
3. **Course**: categorical (nominal)
4. **Season**: categorical (nominal)
5. **Class size**: numeric
6. **Rating**: categorical (ordinal)

Encode the categorical variables in a naive fashion, by leaving them in place as numerics. Run a classification and measure performance against a test set.

In [70]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

fit_and_score_rlr(X, y, normalize=True)

/home/jean/.virtualenvs/learning/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


0.54000000000000004

Now, encode the categorical variables with a one-hot encoder. Again, run a classification and measure performance.

In [71]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(categorical_features=range(5))
X_encoded = enc.fit_transform(X)

fit_and_score_rlr(X_encoded, y, normalize=False)

0.56000000000000005

Comment on your results.

### Feature scaling

Raschka mentions that decision trees and random forests do not require standardized features prior to classification, while the rest of the classifiers we've seen so far do. Why might that be? Explain the intuition behind this idea based on the differences between tree-based classifiers and the other classifiers we've seen.

Now, we'll test the two scaling algorithms on the wine dataset. Start by loading the [wine dataset](https://archive.ics.uci.edu/ml/datasets/Wine).

Scale the features via "standardization" (as Raschka describes it). Classify and measure performance.

Scale the features via "normalization" (as Raschka describes it). Again, classify and measure performance.

Comment on your results.

### Feature selection

- Implement SBS below. Then, run the tests.

In [68]:
class SBS(object):
    """
    Class to select the k-best features in a dataset via sequential backwards selection.
    """
    def __init__(self):
        """
        Initialize the SBS model.
        """
        pass
    
    def fit(self):
        """
        Fit SBS to a dataset.
        """
        pass
    
    def transform(self):
        """
        Transform a dataset based on the model.
        """
        pass
    
    def fit_transform(self):
        """
        Fit SBS to a dataset and transform it, returning the k-best features.
        """
        pass

Now, we'll practice feature selection. Start by loading the breast cancer dataset.

Use a random forest to determine the feature importances. Plot the features and their importances.

Use L1 regularization with a standard C value (0.1) to eliminate low-information features. Again, plot the feature importances using the `coef_` attribute of the model.

How do the feature importances from the random forest/L1 regularization compare?